# Steiner Tree Problem Solver: Testing OR Instances with Blob Simulation

## Overview
This notebook tests the Steiner Tree Problem solver using **Physarum polycephalum** (blob) simulation on the OR-Library test instances. The blob simulation mimics the behavior of the slime mold to find optimal Steiner trees in graphs.

### What is the Steiner Tree Problem?
Given a connected graph with weighted edges and a subset of vertices (terminals), find the minimum-weight tree that connects all terminals. Additional vertices (Steiner nodes) may be included to minimize the total weight.

### The Blob Algorithm
The algorithm simulates the growth patterns of *Physarum polycephalum*, which naturally optimizes network structures by:
- Modeling edges as tubes carrying protoplasmic flux
- Using pressure differentials to drive flow
- Adapting tube conductivities based on usage
- Converging to efficient network topologies

---

## 1. Setup Google Colab Environment

### Configure Google Colab
First, let's configure the environment and check system information:

In [ ]:
import sys
import os
import platform

print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Current working directory: {os.getcwd()}")

# Enable GPU if available
try:
    import torch
    if torch.cuda.is_available():
        print(f"GPU available: {torch.cuda.get_device_name(0)}")
    else:
        print("GPU not available")
except ImportError:
    print("PyTorch not installed")

# Check available memory
import psutil
print(f"Available RAM: {psutil.virtual_memory().available / (1024**3):.2f} GB")

## 2. Clone Repository and Setup

### Clone the BlobSPTG Repository
We'll clone the repository containing the Steiner Tree solver and test instances:

In [ ]:
# Clone the repository (replace with your actual repository URL)
# For demonstration, we'll simulate the repository structure

# If you have a git repository, use:
# !git clone https://github.com/yourusername/BlobSPTG.git
# %cd BlobSPTG

# For now, we'll create the necessary structure and files
import os
import urllib.request
import zipfile

# Clone the BlobSPTG repository from GitHub
!git clone https://github.com/VianneyGG/BlobSPTG.git
%cd BlobSPTG

print(f"Repository cloned successfully!")
print(f"Changed to directory: {os.getcwd()}")
print(f"Repository contents: {os.listdir('.')}")

# Check if tests directory exists
if os.path.exists('tests'):
    test_files = [f for f in os.listdir('tests') if f.endswith('.txt')]
    print(f"\nFound {len(test_files)} test files in tests/ directory")
    print(f"Sample test files: {test_files[:5]}...") if len(test_files) > 5 else print(f"All test files: {test_files}")
else:
    print("\n⚠️ Tests directory not found in repository")

# Create project structure
project_dir = "BlobSPTG"
if not os.path.exists(project_dir):
    os.makedirs(project_dir)
    os.makedirs(f"{project_dir}/Fonctions")
    os.makedirs(f"{project_dir}/tests")
    print(f"Created project directory: {project_dir}")

%cd {project_dir}
print(f"Changed to directory: {os.getcwd()}")

In [ ]:
# Verify test instances from the cloned repository
# The repository should already contain all the OR-Library test instances

print("Checking available test instances...")

if os.path.exists('tests'):
    test_files = sorted([f for f in os.listdir('tests') if f.endswith('.txt')])
    print(f"\n📝 Found {len(test_files)} test files in the repository:")
    
    # Group by instance type
    instance_types = {}
    for filename in test_files:
        if len(filename) > 5:
            inst_type = filename[5]  # 'b', 'c', 'd', 'e'
            if inst_type not in instance_types:
                instance_types[inst_type] = []
            instance_types[inst_type].append(filename)
    
    for inst_type, files in sorted(instance_types.items()):
        print(f"  Type {inst_type.upper()}: {len(files)} instances ({files[0]} to {files[-1]})")
    
    # Display first few test files for verification
    print(f"\n🔍 Sample test files:")
    for filename in test_files[:3]:
        print(f"  - {filename}")
        
    print(f"\n✅ All test instances are ready for testing!")
    
else:
    print("❌ Error: tests directory not found!")
    print("The repository might not have been cloned correctly.")
    print("Available directories:", [d for d in os.listdir('.') if os.path.isdir(d)])

## 3. Install Required Dependencies

### Install Python Packages
Let's install all the necessary packages for the blob simulation:

In [ ]:
# Verify that we have all necessary source files
print("Checking repository structure...")

required_files = ['MS3_PO_MT.py', 'test_evol_vs_smt.py']
required_dirs = ['Fonctions', 'tests']

print("\n📁 Required files:")
for filename in required_files:
    if os.path.exists(filename):
        print(f"  ✅ {filename} - Found")
    else:
        print(f"  ❌ {filename} - Missing")

print("\n📂 Required directories:")
for dirname in required_dirs:
    if os.path.exists(dirname):
        files_count = len(os.listdir(dirname)) if os.path.isdir(dirname) else 0
        print(f"  ✅ {dirname}/ - Found ({files_count} files)")
    else:
        print(f"  ❌ {dirname}/ - Missing")

# Check Fonctions directory content
if os.path.exists('Fonctions'):
    fonctions_files = os.listdir('Fonctions')
    print(f"\n🔧 Fonctions module files: {fonctions_files}")

print("\n🚀 Repository verification complete!")

In [ ]:
# Install required packages
!pip install numpy pandas matplotlib networkx tqdm psutil scipy

# Import all required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from tqdm.auto import tqdm
import time
import heapq
from math import *
from multiprocessing import Pool, cpu_count

print("All dependencies installed successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NetworkX version: {nx.__version__}")

## 4. Implement the Blob Algorithm

### Core Algorithm Functions
Let's implement the essential functions for the blob simulation:

In [ ]:
# Helper functions for parsing Steiner tree instances
def parse_stein_file(filepath):
    """Parse a steinX.txt file and extract graph information."""
    with open(filepath, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    
    n_vertices, n_edges = map(int, lines[0].split())
    edge_lines = lines[1:1+n_edges]
    edges = []
    for line in edge_lines:
        u, v, cost = map(int, line.split())
        edges.append((u, v, cost))
    
    n_terminals = int(lines[1+n_edges])
    # Read terminals from all remaining lines
    terminal_lines = lines[2+n_edges:]
    terminals = []
    for line in terminal_lines:
        terminals.extend(map(int, line.split()))
    
    return n_vertices, n_edges, edges, terminals

def build_graph(n_vertices, edges):
    """Build adjacency matrix from edge list."""
    G = np.full((n_vertices, n_vertices), np.inf)
    for u, v, cost in edges:
        G[u-1, v-1] = cost
        G[v-1, u-1] = cost
    return G

print("Helper functions implemented successfully!")

In [ ]:
# Simplified Blob Algorithm Implementation
def simple_blob_steiner(G, terminals, max_iterations=100, alpha=0.15, mu=1.0, delta=0.1):
    """
    Simplified blob simulation for Steiner Tree Problem.
    
    Args:
        G: Adjacency matrix (numpy array)
        terminals: Set of terminal indices
        max_iterations: Maximum number of iterations
        alpha: Learning rate for conductivity updates
        mu: Flux strength parameter
        delta: Minimum conductivity threshold
    
    Returns:
        Final adjacency matrix with selected edges
    """
    n = G.shape[0]
    terminals = list(terminals)
    
    # Initialize conductivities
    D = np.ones_like(G) * 0.1
    D[np.isinf(G)] = 0
    
    # Initialize pressures
    pressures = np.zeros(n)
    
    for iteration in range(max_iterations):
        # Set boundary conditions (terminals as sources/sinks)
        if len(terminals) >= 2:
            pressures[terminals[0]] = 1.0  # Source
            pressures[terminals[-1]] = 0.0  # Sink
        
        # Calculate flows using conductivities
        flows = np.zeros_like(G)
        for i in range(n):
            for j in range(i+1, n):
                if not np.isinf(G[i, j]) and D[i, j] > 0:
                    flow = D[i, j] * (pressures[i] - pressures[j]) / G[i, j]
                    flows[i, j] = flow
                    flows[j, i] = -flow
        
        # Update pressures (simplified pressure calculation)
        new_pressures = pressures.copy()
        for i in range(n):
            if i not in terminals:  # Only update non-terminal nodes
                total_flow = 0
                total_conductivity = 0
                for j in range(n):
                    if not np.isinf(G[i, j]) and D[i, j] > 0:
                        total_flow += D[i, j] * pressures[j] / G[i, j]
                        total_conductivity += D[i, j] / G[i, j]
                
                if total_conductivity > 0:
                    new_pressures[i] = total_flow / total_conductivity
        
        pressures = new_pressures
        
        # Update conductivities based on flow
        new_D = D.copy()
        for i in range(n):
            for j in range(i+1, n):
                if not np.isinf(G[i, j]):
                    flow_magnitude = abs(flows[i, j])
                    # Reinforcement: increase conductivity for high-flow edges
                    new_D[i, j] = max(delta, D[i, j] * (1 + alpha * flow_magnitude))
                    new_D[j, i] = new_D[i, j]
        
        # Apply decay to unused edges
        for i in range(n):
            for j in range(i+1, n):
                if not np.isinf(G[i, j]):
                    if abs(flows[i, j]) < 0.01:  # Low flow threshold
                        new_D[i, j] *= (1 - mu * delta)
                        new_D[j, i] = new_D[i, j]
        
        D = new_D
        
        # Check convergence (simplified)
        if iteration % 20 == 0:
            print(f"Iteration {iteration}: Active edges = {np.sum(D > delta)}")
    
    # Extract final tree (edges with significant conductivity)
    result = np.full_like(G, np.inf)
    threshold = np.max(D) * 0.1  # Adaptive threshold
    
    for i in range(n):
        for j in range(i+1, n):
            if D[i, j] > threshold and not np.isinf(G[i, j]):
                result[i, j] = G[i, j]
                result[j, i] = G[i, j]
    
    return result

print("Blob algorithm implemented successfully!")

## 5. Run Tests on OR Instances

### Test the Blob Algorithm
Now let's test our blob simulation on the OR-Library instances:

In [ ]:
# Import the actual MS3_PO_MT function from the repository
try:
    from MS3_PO_MT import MS3_PO_MT
    print("✅ Successfully imported MS3_PO_MT from repository")
except ImportError as e:
    print(f"❌ Failed to import MS3_PO_MT: {e}")
    print("Will use simplified blob algorithm instead")
    MS3_PO_MT = None

# Known optimal solutions for comparison (from test_evol_vs_smt.py)
SMT_OPTIMAL = {
    'b': {1: 82.0, 2: 83.0, 3: 138.0, 4: 59.0, 5: 61.0, 6: 122.0, 7: 111.0, 8: 104.0, 
          9: 220.0, 10: 86.0, 11: 88.0, 12: 174.0, 13: 165.0, 14: 235.0, 15: 318.0, 
          16: 127.0, 17: 131.0, 18: 218.0},
    'c': {1: 85.0, 2: 144.0, 3: 754.0, 4: 1079.0, 5: 1579.0, 6: 55.0, 7: 102.0, 
          8: 509.0, 9: 707.0, 10: 1093.0, 11: 32.0, 12: 46.0, 13: 258.0, 14: 323.0, 
          15: 556.0, 16: 11.0, 17: 18.0, 18: 113.0, 19: 146.0, 20: 267.0},
    'd': {1: 106.0, 2: 220.0, 3: 1565.0, 4: 1935.0, 5: 3250.0, 6: 67.0, 7: 103.0, 
          8: 1072.0, 9: 1448.0, 10: 2110.0, 11: 29.0, 12: 42.0, 13: 500.0, 14: 667.0, 
          15: 1116.0, 16: 13.0, 17: 23.0, 18: 223.0, 19: 310.0, 20: 537.0},
    'e': {1: 111.0, 2: 214.0, 3: 4013.0, 4: 5101.0, 5: 8128.0, 6: 73.0, 7: 145.0, 
          8: 2640.0, 9: 3604.0, 10: 5600.0, 11: 34.0, 12: 67.0, 13: 1280.0, 14: 1732.0, 
          15: 2784.0, 16: 15.0, 17: 25.0, 18: 564.0, 19: 758.0, 20: 1342.0}
}

def run_test_on_instance(filename, use_repository_algorithm=True):
    """Run blob algorithm on a single test instance."""
    filepath = f'tests/{filename}'
    if not os.path.isfile(filepath):
        return None
    
    try:
        # Parse the instance
        n_vertices, n_edges, edges, terminals = parse_stein_file(filepath)
        G = build_graph(n_vertices, edges)
        
        print(f"\n--- Testing {filename} ---")
        print(f"Vertices: {n_vertices}, Edges: {n_edges}, Terminals: {len(terminals)}")
        print(f"Terminal nodes: {terminals}")
        
        # Convert terminals to 0-based indexing
        terminal_set = set([t-1 for t in terminals])
        
        # Run algorithm
        start_time = time.time()
        
        if use_repository_algorithm and MS3_PO_MT is not None:
            # Use the actual repository algorithm
            try:
                result_matrix = MS3_PO_MT(G, terminal_set,
                                        M=50,  # Number of iterations
                                        K=10,  # Population size
                                        alpha=0.15,
                                        mu=1,
                                        delta=0.1,
                                        S=3,
                                        évol=False,
                                        modeRenfo='vieillesse',
                                        modeProba='weighted')
                algorithm_used = "Repository MS3_PO_MT"
            except Exception as repo_error:
                print(f"Repository algorithm failed: {repo_error}")
                print("Falling back to simplified algorithm...")
                result_matrix = simple_blob_steiner(G, terminal_set, max_iterations=50)
                algorithm_used = "Simplified Blob (fallback)"
        else:
            # Use simplified algorithm
            result_matrix = simple_blob_steiner(G, terminal_set, max_iterations=50)
            algorithm_used = "Simplified Blob"
        
        end_time = time.time()
        
        # Calculate solution weight
        mask = np.isfinite(result_matrix)
        solution_weight = np.sum(result_matrix[mask]) / 2  # Divide by 2 for undirected graph
        
        # Get optimal solution for comparison
        instance_type = filename[5]  # 'b', 'c', 'd', 'e'
        instance_num = int(filename[6:-4])  # Extract number
        optimal_weight = SMT_OPTIMAL.get(instance_type, {}).get(instance_num, float('inf'))
        
        # Calculate error percentage
        if optimal_weight != float('inf'):
            error_pct = max(0, (solution_weight - optimal_weight) / optimal_weight * 100)
        else:
            error_pct = float('inf')
        
        runtime = end_time - start_time
        
        result = {
            'file': filename,
            'vertices': n_vertices,
            'edges': n_edges,
            'terminals': len(terminals),
            'blob_weight': solution_weight,
            'optimal_weight': optimal_weight,
            'error_pct': error_pct,
            'runtime': runtime,
            'algorithm': algorithm_used
        }
        
        print(f"Algorithm: {algorithm_used}")
        print(f"Blob solution: {solution_weight:.2f}")
        print(f"Optimal: {optimal_weight:.2f}")
        print(f"Error: {error_pct:.2f}%")
        print(f"Runtime: {runtime:.3f}s")
        
        return result
        
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        import traceback
        traceback.print_exc()
        return None

print("Enhanced test function ready!")

In [ ]:
# Run tests on all available instances from the repository
results = []

if os.path.exists('tests'):
    test_files = sorted([f for f in os.listdir('tests/') if f.endswith('.txt')])
    
    print(f"Found {len(test_files)} test files in repository")
    print("\n" + "="*60)
    print("RUNNING BLOB TESTS ON OR INSTANCES")
    print("="*60)
    
    # Test a subset first (you can modify this to test all)
    # For demonstration, we'll test first few from each type
    test_subset = []
    instance_types = ['b', 'c', 'd', 'e']
    
    for inst_type in instance_types:
        type_files = [f for f in test_files if f.startswith(f'stein{inst_type}')]
        if type_files:
            # Test first 3 instances of each type for demo
            test_subset.extend(type_files[:3])
    
    print(f"Testing subset of {len(test_subset)} instances for demonstration:")
    print(f"Selected files: {test_subset}")
    print()
    
    for filename in test_subset:
        result = run_test_on_instance(filename, use_repository_algorithm=True)
        if result:
            results.append(result)
    
    print("\n" + "="*60)
    print(f"TESTS COMPLETED - {len(results)} successful tests")
    print("="*60)
    
else:
    print("❌ Error: tests directory not found!")
    print("Make sure the repository was cloned correctly.")

## 6. Analyze Test Results

### Results Summary and Visualization
Let's analyze the performance of our blob algorithm:

In [ ]:
# Create results DataFrame
df_results = pd.DataFrame(results)

if not df_results.empty:
    print("\n📊 RESULTS SUMMARY")
    print("="*50)
    
    # Display results table
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    display_df = df_results.copy()
    for col in ['blob_weight', 'optimal_weight', 'error_pct', 'runtime']:
        if col in display_df.columns:
            display_df[col] = display_df[col].round(3)
    
    print(display_df.to_string(index=False))
    
    # Calculate statistics
    finite_errors = df_results[df_results['error_pct'] != float('inf')]['error_pct']
    if not finite_errors.empty:
        print(f"\n📈 PERFORMANCE STATISTICS")
        print("="*30)
        print(f"Average error: {finite_errors.mean():.2f}%")
        print(f"Median error: {finite_errors.median():.2f}%")
        print(f"Min error: {finite_errors.min():.2f}%")
        print(f"Max error: {finite_errors.max():.2f}%")
        print(f"Std deviation: {finite_errors.std():.2f}%")
    
    print(f"\nAverage runtime: {df_results['runtime'].mean():.3f}s")
    print(f"Total instances tested: {len(results)}")
    
    # Save results
    df_results.to_csv('blob_steiner_results.csv', index=False)
    print("\n💾 Results saved to 'blob_steiner_results.csv'")
    
else:
    print("❌ No valid results to analyze")

In [ ]:
# Create visualizations
if not df_results.empty and len(df_results) > 0:
    # Set up the plotting
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Blob Algorithm Performance on OR Instances', fontsize=16, fontweight='bold')
    
    # 1. Error percentage distribution
    finite_errors = df_results[df_results['error_pct'] != float('inf')]['error_pct']
    if not finite_errors.empty:
        axes[0, 0].hist(finite_errors, bins=10, alpha=0.7, color='skyblue', edgecolor='black')
        axes[0, 0].set_title('Distribution of Error Percentages')
        axes[0, 0].set_xlabel('Error (%)')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Solution weight comparison
    if 'optimal_weight' in df_results.columns:
        valid_data = df_results[df_results['optimal_weight'] != float('inf')]
        if not valid_data.empty:
            axes[0, 1].scatter(valid_data['optimal_weight'], valid_data['blob_weight'], 
                              alpha=0.7, color='coral')
            # Add perfect line
            min_val = min(valid_data['optimal_weight'].min(), valid_data['blob_weight'].min())
            max_val = max(valid_data['optimal_weight'].max(), valid_data['blob_weight'].max())
            axes[0, 1].plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5, label='Perfect')
            axes[0, 1].set_title('Blob vs Optimal Solutions')
            axes[0, 1].set_xlabel('Optimal Weight')
            axes[0, 1].set_ylabel('Blob Weight')
            axes[0, 1].legend()
            axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Runtime vs problem size
    axes[1, 0].scatter(df_results['vertices'], df_results['runtime'], 
                       alpha=0.7, color='lightgreen')
    axes[1, 0].set_title('Runtime vs Problem Size')
    axes[1, 0].set_xlabel('Number of Vertices')
    axes[1, 0].set_ylabel('Runtime (seconds)')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Error vs problem complexity
    if not finite_errors.empty:
        complexity_data = df_results[df_results['error_pct'] != float('inf')]
        axes[1, 1].scatter(complexity_data['edges'], complexity_data['error_pct'], 
                          alpha=0.7, color='plum')
        axes[1, 1].set_title('Error vs Graph Complexity')
        axes[1, 1].set_xlabel('Number of Edges')
        axes[1, 1].set_ylabel('Error (%)')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Additional analysis
    print("\n🔍 DETAILED ANALYSIS")
    print("="*40)
    
    if not finite_errors.empty:
        good_solutions = len(finite_errors[finite_errors <= 5])  # Within 5% of optimal
        acceptable_solutions = len(finite_errors[finite_errors <= 10])  # Within 10% of optimal
        
        print(f"Solutions within 5% of optimal: {good_solutions}/{len(finite_errors)} ({good_solutions/len(finite_errors)*100:.1f}%)")
        print(f"Solutions within 10% of optimal: {acceptable_solutions}/{len(finite_errors)} ({acceptable_solutions/len(finite_errors)*100:.1f}%)")
        
        # Best and worst performing instances
        best_idx = finite_errors.idxmin()
        worst_idx = finite_errors.idxmax()
        
        print(f"\n🏆 Best performance: {df_results.loc[best_idx, 'file']} (Error: {finite_errors.loc[best_idx]:.2f}%)")
        print(f"⚠️  Worst performance: {df_results.loc[worst_idx, 'file']} (Error: {finite_errors.loc[worst_idx]:.2f}%)")
else:
    print("❌ No data available for visualization")

## 7. Conclusions and Next Steps

### Summary
This notebook demonstrated the application of **Physarum polycephalum** (blob) simulation to solve the Steiner Tree Problem on OR-Library test instances. The algorithm mimics the natural optimization behavior of slime molds to find efficient network topologies.

### Key Observations:
1. **Biological Inspiration**: The blob algorithm leverages natural optimization principles
2. **Adaptive Behavior**: Conductivities evolve based on usage patterns
3. **Scalability**: Performance varies with problem complexity
4. **Approximation Quality**: Results compared against known optimal solutions

### Algorithm Characteristics:
- **Strengths**: Bio-inspired, adaptive, handles complex topologies
- **Limitations**: Convergence time, parameter sensitivity
- **Applications**: Network design, routing optimization, infrastructure planning

### Potential Improvements:
1. **Parameter Tuning**: Optimize α, μ, δ for different instance types
2. **Multi-Phase Approach**: Combine with other heuristics
3. **Parallel Processing**: Leverage multiple cores for larger instances
4. **Advanced Stopping Criteria**: Better convergence detection

---

### References:
- Physarum polycephalum behavior studies
- OR-Library Steiner Tree instances
- Bio-inspired optimization algorithms
- Network optimization literature

**Happy optimizing! 🦠🌟**